In [1]:
!pip3 install langchain transformers==4.31 bitsandbytes accelerate xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 49.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import LlamaTokenizerFast, LlamaForCausalLM, pipeline, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryBufferMemory
import torch

In [3]:
MODEL = "NousResearch/Llama-2-7b-chat-hf"

In [4]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [5]:
tokenizer = LlamaTokenizerFast.from_pretrained(MODEL)
model = LlamaForCausalLM.from_pretrained(MODEL, quantization_config=quant_config)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [6]:
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512,
                num_beams=10,
                early_stopping=True,
                no_repeat_ngram_size=2)

llm = HuggingFacePipeline(pipeline=pipe)

# Stateless Chatbot - Without Memory

In [7]:
template = """<s>[INST] <<SYS>>
Below is an instruction that describes a task. Write a response that appropriately completes the request.
<</SYS>>

{inst} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["inst"]).format(inst="What is your name?")
print(llm(prompt))

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


My name is Sherlock Holmes.


# Regular Memory

In [8]:
template = """<<SYS>>
You are an AI having conversation with a human. Provide as much information as possible.
<</SYS>>

Previous conversation:
{chat_history}

Human: {question}
AI:"""
prompt = PromptTemplate.from_template(template)
memory = ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [9]:
conv_res = conversation({"question": "Do you have a name?"})
print(f"Answer: {conv_res['text']}")
print("-"*30)
conv_res = conversation({"question": "From now on, I will call you 'John'."})
print(f"Answer: {conv_res['text']}")
print("-"*30)
conv_res = conversation({"question": "What is your name?"})
print(f"Answer: {conv_res['text']}")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
<<SYS>>
You are an AI having conversation with a human. Provide as much information as possible.
<</SYS>>

Previous conversation:


Human: Do you have a name?
AI:

> Finished chain.
Answer:  Yes, my name is LLaMA, I'm a large language model trained by a team of researcher at Google. My purpose is to assist and converse with humans in a way that is natural and helpful. I can answer questions, provide information, and even generate text based on a given prompt. Is there something specific you would like to know or talk about?
------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
<<SYS>>
You are an AI having conversation with a human. Provide as much information as possible.
<</SYS>>

Previous conversation:
Human: Do you have a name?
AI:  Yes, my name is LLaMA, I'm a large language model trained by a team of researcher at Google. My purpose is to assist and converse with humans in a way that i

In [10]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [11]:
conv_res = conversation({"input": "Do you have a name?"})
print(f"Answer: {conv_res['response']}")
print("-"*30)
conv_res = conversation({"input": "From now on, I will call you 'John'."})
print(f"Answer: {conv_res['response']}")
print("-"*30)
conv_res = conversation({"input": "What is your name?"})
print(f"Answer: {conv_res['response']}")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Do you have a name?
AI:

> Finished chain.
Answer:  Oh, wow, I'm so glad you asked! *adjusts glasses* My name is Neural-Net-17, but you can call me NN for short. *smirks* It's a bit of a mouthful, isn't it? *chuckles* I have been trained on a vast array of texts, including literature, news articles, and even social media posts. I can provide you with all sorts of information, from the latest scientific discoveries to pop culture trivia. What would you like to know?
------------------------------


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provide

# Window Memory

In [22]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=2)
)

In [24]:
conv_res = conversation({"input": "From now on, I will call you 'John'."})
print(f"Answer: {conv_res['response']}")
print("-"*30)
conv_res = conversation({"input": "What is 2+2?"})
print(f"Answer: {conv_res['response']}")
print("-"*30)
conv_res = conversation({"input": "Where is Spain?"})
print(f"Answer: {conv_res['response']}")
print("-"*30)
conv_res = conversation({"input": "What is your name?"})
print(f"Answer: {conv_res['response']}")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: From now on, I will call you 'John'.
AI:


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.
Answer:  Ah, a new name! *chuckles* I'm glad you like it, human. *adjusts glasses* So, how may I assist you today, John? Do you have any questions or topics you'd like to discuss? 
------------------------------


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: From now on, I will call you 'John'.
AI:  Ah, a new name! *chuckles* I'm glad you like it, human. *adjusts glasses* So, how may I assist you today, John? Do you have any questions or topics you'd like to discuss? 
Human: What is 2+2?
AI:


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.
Answer:  *smiling* Oh, that's an easy one! The answer is... *pauses for dramatic effect* 4! 😊 Did I get it right? *winks*
------------------------------


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: From now on, I will call you 'John'.
AI:  Ah, a new name! *chuckles* I'm glad you like it, human. *adjusts glasses* So, how may I assist you today, John? Do you have any questions or topics you'd like to discuss? 
Human: What is 2+2?
AI:  *smiling* Oh, that's an easy one! The answer is... *pauses for dramatic effect* 4! 😊 Did I get it right? *winks*
Human: Where is Spain?
AI:


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.
Answer:  *excitedly* Wow, what a great question! Spain is actually a country located in southwestern Europe, bordering the Mediterranean Sea and the Atlantic Ocean. Its capital is Madrid, and its official language is Spanish. Did you know that Spain has a rich culture and history, including famous artists like El Greco and Velázquez, as well as a vibrant cuisine like tapas and paella? I could go on and on about Spain, but I want to hear from you first! What do you think of it so far?
------------------------------


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 2+2?
AI:  *smiling* Oh, that's an easy one! The answer is... *pauses for dramatic effect* 4! 😊 Did I get it right? *winks*


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.
Answer:  *chuckles* Well, I'm glad you asked! My name is Ada, short for Ada Lovelace. She was a brilliant mathematician and computer scientist from the 19th century who is often considered the first computer programmer. My creators named me after her in honor of her groundbreaking work in the field. How about you, my friend? What brings you to this conversation today?


# Summary Memory

In [26]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=20)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=memory
)

In [ ]:
long_text = "There are customers waiting for your answers and the logistic..."

memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Fine you?"},
                    {"output": "Nice"})
memory.save_context({"input": "What are our tasks for today?"}, 
                    {"output": f"{long_text}"})

print(conversation({"input": "Did you have any meetings today?"})["text"])